In [19]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import yfinance as yf
import keras.backend as K



In [20]:
# Define the quantile loss function
def quantile_loss(q, y_true, y_pred):
    e = y_true - y_pred
    return K.mean(K.maximum(q * e, (q - 1) * e), axis=-1)


In [21]:
# Load the dataset
msft = yf.Ticker("^GDAXI")
hist = msft.history(period="max")

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(hist['Close'].values.reshape(-1,1))



In [22]:
# Define the time step and create the dataset in the required format
time_step = 60
X = []
y = []

for i in range(time_step, len(scaled_data)):
    X.append(scaled_data[i-time_step:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))






In [23]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [24]:
# Define the LSTM model architecture
model = Sequential()

# Stacked LSTM layers with Dropout
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))

# Dense layer for output
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=lambda y_true, y_pred: quantile_loss(0.5, y_true, y_pred))



2023-11-05 11:06:16.543557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-05 11:06:16.544678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-05 11:06:16.545385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [27]:
# Cell 6: Train the Model
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/5
226/226 [==============================] - 14s 63ms/step - loss: 0.0102 - val_loss: 0.0039
Epoch 2/5
226/226 [==============================] - 14s 62ms/step - loss: 0.0101 - val_loss: 0.0064
Epoch 3/5
226/226 [==============================] - 14s 62ms/step - loss: 0.0104 - val_loss: 0.0056
Epoch 4/5
226/226 [==============================] - 14s 61ms/step - loss: 0.0100 - val_loss: 0.0050
Epoch 5/5
226/226 [==============================] - 14s 62ms/step - loss: 0.0099 - val_loss: 0.0050


In [28]:
# Predict and compute the quantiles
predictions = model.predict(X_test)
quantiles = np.quantile(predictions, [0.025, 0.25, 0.5, 0.75, 0.975], axis=0)


2023-11-05 11:10:12.213421: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-05 11:10:12.214866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-05 11:10:12.215686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

57/57 [==============================] - 1s 12ms/step


In [35]:
predictions.shape

(1801, 1)

In [32]:
# Assuming quantiles is a 1D array with 5 elements, one for each quantile
# And assuming you want the same quantile values for each of the 5 horizons

# Create the submission DataFrame
# Assuming predictions is a 2D array with shape (5, number_of_quantiles), where the first dimension is the horizon
df_sub = pd.DataFrame({
    "target": "DAX",
    "horizon": [str(i) + " day" for i in (1, 2, 5, 6, 7)],
    "q0.025": predictions[:, 0],  # Use the first quantile for each horizon
    "q0.25": predictions[:, 1],   # Use the second quantile for each horizon
    "q0.5": predictions[:, 2],    # And so on...
    "q0.75": predictions[:, 3],
    "q0.975": predictions[:, 4]
})


# Save the DataFrame to a CSV file
df_sub.to_csv("DAX_quantile_predictions2.csv", index=False)



IndexError: index 1 is out of bounds for axis 1 with size 1